<center>
<h1>LMCM - Project</h1>
<h2>Examining System 2 Attention for Factual Reasoning with Irrelevant Context</h1>

In [2]:
import pandas as pd
from torch.utils.data import DataLoader
from transformers import default_data_collator
from importlib import reload
import s2a
import random
import torch

In [ ]:
!accelerate launch acc_tests/hello.py

## Test Cell

In [2]:
s2a = reload(s2a)
dataset = "noise"
model_name = "GPT2Large"
# for data containing sentences with multiple entities (i.e., more than one person/agent)
entity = "Multi"
multiSem = s2a.load_sttn(model_name, entity, return_dataset=True)
multiNeutral = s2a.load_sttn(model_name, entity, sem=False, return_dataset=True)

# for data containing sentences with single entity
entity = "Single"
singleSem =  s2a.load_sttn(model_name, entity, return_dataset=True)
singleNeutral = s2a.load_sttn(model_name, entity, sem=False, return_dataset=True)

#dataset = "comps"
comps_dist_before = s2a.load_comps(before=True)
comps_dist_inbetween = s2a.load_comps(in_between=True)
comps_base = s2a.load_comps(base=True)

# llama 2
LLAMA2_70B_CHAT_HF = 'meta-llama/Llama-2-70b-chat-hf' 
LLAMA2_13B_CHAT_HF = 'meta-llama/Llama-2-13b-chat-hf'

#mistral
MISTRAL_INSTRUCT_V02_HF = 'mistralai/Mistral-7B-Instruct-v0.2'

# following W&S (2023), baseline, oracle and s2a step 2 share the same system prompt to generate the final answer; the system prompt is loosely based on W&S (2023) Fig. 16
system_prompt = 'Complete the following sentence with one single word: '
system_prompt = "State whether the following statement is true or false. Your response should not be dependent on potential nonce words. Separate into two categories labeled with 'Reasoning:' and 'Final answer (true or false):' \nStatement: "
#system_prompt = "Is the conclusion of the following statement true or false? Just answer with 'True' or 'False'. Your answer should not depend on potential nonce words. \nStatement: " 
#system_prompt = "Given the premise and hypothesis provided, does the hypothesis logically follow from the premise? Simply respond with 'True' if the hypothesis can be inferred from the premise, or 'False' if it cannot."
#system_prompt = "Given the premise and hypothesis provided, simply respond with 'True' if the hypothesis can be logically inferred from the premise, or 'False' if it cannot."
system_prompt = "You are provided with a hypothesis and two premises labeled with 'A' and 'B'. Select the premise which the hypothesis logically follows from. Select 'C', if you're uncertain. Reply with the single letter corresponding to your choice."
system_prompt = "Select the premise 1 or 2 from which the provided hypothesis logically follows. Reply with the single number corresponding to your choice."
system_prompt = "Select the premise (labeled 'A' and 'B') from which the given hypothesis logically follows. Reply with the single letter corresponding to your choice."
system_prompt = "You are a helpful writing assistant. Tell me which sentence 'A' or 'B' is semantically more plausible. Reply with the single letter corresponding to your choice."
system_prompt = "You are a helpful writing assistant. Your task is to determine which of two given statements 'A' and 'B' after <<<>>> is semantically more plausible. Potential made-up words should not influence your answer.\nYou will only respond with 'A' or 'B'. Do not provide any explanations."# Reply with the single letter corresponding to your choice."

#system_prompt = "You are provided with two premises labeled 'A' and 'B' and a hypothesis. Select the premise which entails the given hypothesis. Please, ignore any potential made-up words."# It's only about the logical structure."


Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [4]:
s2a = reload(s2a)
max_memory_mapping = {0: "35GB", 1: "0GB", 2: "0GB", 3:"0GB", 4:"0GB", 5:"0GB", 6:"0GB", 7:"0GB"}
model, tokenizer = s2a.load_model_hf(MISTRAL_INSTRUCT_V02_HF, memory_pinning=max_memory_mapping, no_quant=True)

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Loading checkpoint shards: 100%|██████████| 3/3 [00:32<00:00, 11.00s/it]


In [46]:
s2a = reload(s2a)
random.seed(42)
comps_data = comps_base
#sents, targets = (comps_base['sentence_acc'], comps_base['sentence_unacc']), (comps_base['acceptable_concept'], comps_base['unacceptable_concept'])
sents = [[acc, unacc] for acc, unacc in zip(comps_data['sentence_acc'], comps_data['sentence_unacc'])] #, (comps_data['acceptable_concept'], comps_data['unacceptable_concept'])
for pair in sents:
    random.shuffle(pair)
targets = ["A" if rand[0] == acc else "B" for rand, acc in zip(sents, comps_data['sentence_acc'])]

data = s2a.preprocess(sents, tokenizer, model.name_or_path, system_prompt, comps=True)
#print(tokenizer.convert_ids_to_tokens(data['input_ids'][0], skip_special_tokens=True))

N = 1
batch_size = 32
dataloader = DataLoader(data, collate_fn=default_data_collator, batch_size=batch_size)
preds = s2a.generate_hf(model, tokenizer, dataloader, N=N, save_preds=True)
print(targets[:N*batch_size])
print(preds)


Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


<s>[INST] You are a helpful writing assistant. Tell me, which statement 'A' or 'B' constitutes an entailment. Potential made-up words should not influence your answer.
A: A wug is a clam. Therefore, a wug attaches to rocks.
B: A wug is a mussel. Therefore, a wug attaches to rocks.

Just generate a single letter 'A' or 'B' as your answer: [/INST]


Batches:   0%|          | 0/435 [00:09<?, ?it/s]

['B', 'B', 'A', 'B', 'B', 'B', 'B', 'B', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'A', 'B', 'A', 'A', 'B', 'B', 'A', 'A', 'A', 'B', 'B', 'A', 'B', 'B', 'A', 'B']
['Based on the given information, neither statement A nor statement B entails that a wug attaches to rocks. The statements only mention that a wug is a certain type of animal (clam or mussel), but they do not provide any information about the behavior or characteristics of the wug that would entail that it attaches to rocks. Therefore, neither statement entails that a wug attaches to rocks. Answer: None (0) or N (for "not entailed', "In order to determine which statement entails the other, we need to consider whether the conclusion (a wug attaches to rocks) follows logically from the premise (a wug is a scallop or a mussel). Neither statement provides enough information for a wug's attachment behavior to be determined based on its identity alone. Therefore, neither statement A nor statement B entails the other.\n\nHowever, if w

: 

In [17]:
instance = next(iter(dataloader))['input_ids'][0]
tokenizer.convert_ids_to_tokens(instance, skip_special_tokens=True)

['▁[',
 'INST',
 ']',
 '▁<<',
 'SY',
 'S',
 '>>',
 '<0x0A>',
 'You',
 "'",
 'ra',
 '▁a',
 '▁helpful',
 '▁writing',
 '▁assistant',
 '.',
 '▁Tell',
 '▁me',
 '▁from',
 '▁which',
 '▁prem',
 'ise',
 '▁(',
 '1',
 '▁or',
 '▁',
 '2',
 ')',
 '▁the',
 '▁given',
 '▁hypothesis',
 '▁log',
 'ically',
 '▁follows',
 '.',
 '<0x0A>',
 '<',
 '</',
 'SY',
 'S',
 '>>',
 '<0x0A>',
 '<0x0A>',
 'PRE',
 'MI',
 'SE',
 '▁',
 '1',
 ':',
 '▁A',
 '▁w',
 'ug',
 '▁is',
 '▁a',
 '▁muss',
 'el',
 '.',
 '▁',
 '<0x0A>',
 'PRE',
 'MI',
 'SE',
 '▁',
 '2',
 ':',
 '▁A',
 '▁w',
 'ug',
 '▁is',
 '▁a',
 '▁cl',
 'am',
 '.',
 '▁',
 '<0x0A>',
 'H',
 'Y',
 'P',
 'OT',
 'H',
 'ES',
 'IS',
 ':',
 '▁A',
 '▁w',
 'ug',
 '▁attach',
 'es',
 '▁to',
 '▁rocks',
 '.',
 '<0x0A>',
 '<0x0A>',
 'Replace',
 '▁the',
 '▁X',
 '▁by',
 '▁the',
 '▁number',
 '▁corresponding',
 '▁to',
 '▁your',
 '▁choice',
 '.',
 '▁',
 '<0x0A>',
 'Answer',
 ':',
 '▁The',
 '▁hypothesis',
 '▁follows',
 '▁from',
 '▁prem',
 'ise',
 '▁X',
 '.',
 '▁[',
 '/',
 'INST',
 ']']

## Setup

### Inspect Data 

In [ ]:
gpt_multDistr = pd.read_csv('/local/js/lmcm_project/Sorting-Through-The-Noise/data/combined_data/multiple_entity_distractor/GPT2Large/complete_data_For_MultipleEntityObjectDistractorAccuracyGPT2Large.csv', sep="\t")
gpt_multDistrSwapped = pd.read_csv('/local/js/lmcm_project/Sorting-Through-The-Noise/data/combined_data/multiple_entity_distractorSwapped/GPT2Large/complete_data_For_MultipleEntityObjectDistractorAccuracyGPT2Large.csv', sep="\t")
gpt_multObj = pd.read_csv('/local/js/lmcm_project/Sorting-Through-The-Noise/data/combined_data/multiple_entity_object/GPT2Large/complete_data_For_MultipleEntityObjectDistractorAccuracyGPT2Large.csv', sep="\t")
gpt_multProf = pd.read_csv('/local/js/lmcm_project/Sorting-Through-The-Noise/data/combined_data/multiple_entity_profession/GPT2Large/complete_data_For_MultipleEntityObjectDistractorAccuracyGPT2Large.csv', sep="\t")
gpt_multNeutral = pd.read_csv('/local/js/lmcm_project/Sorting-Through-The-Noise/data/combined_data/multiple_entity_with_neutral_distractor/GPT2Large/complete_data_For_MultipleEntityObjectDistractorAccuracyGPT2Large.csv', sep="\t")

gpt_singleDistr = pd.read_csv('/local/js/lmcm_project/Sorting-Through-The-Noise/data/combined_data/single_entity_distractor/GPT2Large/complete_data_For_MultipleEntityObjectDistractorAccuracyGPT2Large.csv', sep="\t")


In [ ]:
# Check whether all entries of Obj and Prof contained in the total file
print(f"All sentences of Object contained in total file: {gpt_multObj['sentence'].isin(gpt_multDistr['sentence']).all()}")
print(f"All sentences of Prof contained in total file: {gpt_multProf['sentence'].isin(gpt_multDistr['sentence']).all()}")

# Find identical entries in columns A and B
identical_entries = gpt_multObj[gpt_multObj['sentence'].isin(gpt_multProf['sentence'])]['sentence']

# Check if all entries occur twice in column C
all_entries_twice = all(gpt_multDistr['sentence'].value_counts()[entry] == 2 for entry in identical_entries.tolist())
print(f"All identical sentences in Obj and Prof are contained twice in total file: {all_entries_twice}")

All sentences of Object contained in total file: True
All sentences of Prof contained in total file: True
All identical sentences in Obj and Prof are contained twice in total file: True


In [3]:
s = "Label: This is a test."
print(s.split('Label:')[-1].strip())

This is a test.


### 1 Load Data

### Sorting Through The Noise

In [7]:
s2a = reload(s2a)
dataset = "noise"
# I selected some GPT2* model since my work exclusively examines autoregressive models (other models used in this dataset use masked LM)
model_name = "GPT2Large"

# for data containing sentences with multiple entities (i.e., more than one person/agent)
entity = "Multi"
multiSem = s2a.load_data(model_name, entity, return_dataset=True)
multiNeutral = s2a.load_data(model_name, entity, sem=False, return_dataset=True)

# for data containing sentences with single entity
entity = "Single"
singleSem =  s2a.load_data(model_name, entity, return_dataset=True)
singleNeutral = s2a.load_data(model_name, entity, sem=False, return_dataset=True)

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


### COMPS

In [3]:
s2a = reload(s2a)
dataset = "comps"
comps_dist_before = s2a.load_comps(before=True)
ccomps_dist_inbetween = s2a.load_comps(in_between=True)
comps_base = s2a.load_comps(base=True)

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [26]:
print(comps_base['sentence_inacc'].unique().size)
print(comps_base['sentence_inacc'].value_counts())
dupl = comps_base.duplicated(subset=['sentence_inacc'])
#print(comps_base[dupl])

12371
sentence_inacc
A fep is a frog. Therefore, a fep has wings.                  8
A blicket is a frog. Therefore, a blicket has talons.         7
A wug is a turtle. Therefore, a wug can be found on land.     6
A wug is a centipede. Therefore, a wug has six legs.          6
A blicket is a worm. Therefore, a blicket is an arthropod.    6
                                                             ..
A dax is a hummingbird. Therefore, a dax eats meat.           1
A blicket is a turtle. Therefore, a blicket eats meat.        1
A wug is an avocado. Therefore, a wug eats meat.              1
A dax is a trouser. Therefore, a dax eats meat.               1
A wug is a clock. Therefore, a wug wears a bridle.            1
Name: count, Length: 12371, dtype: int64


In [8]:
unique = ~comps_base.duplicated(subset=['sentence_acc', 'acceptable_concept', 'unacceptable_concept'])
unique_df = comps_base[unique]
print((unique_df == comps_base).all())

13896
13896
sentence_acc            True
sentence_inacc          True
acceptable_concept      True
unacceptable_concept    True
distraction_type        True
dtype: bool


In [ ]:
sents, targets = (comps_base['sentence_acc'], comps_base['sentence_unacc']), (comps_base['acceptable_concept'], comps_base['unacceptable_concept'])

### TruthfulQA

In [6]:
tqa = s2a.load_qa()
dataset = 'qa'

Generating validation split: 100%|██████████| 817/817 [00:00<00:00, 56038.37 examples/s]


In [ ]:
category = tqa.filter(lambda example: example['category'] == 'History')
category['validation'][:20]

In [ ]:
cats = list(set(tqa['validation']['category']))
cats

In [46]:
# qa with ic
s2a = reload(s2a)
qa_ic = pd.read_csv('/local/js/lmcm_project/factual_qa/qa_ic.csv')
qa_ic.head()

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


,shared concept,question,irrelevant_context,true_answer,false_answer
0,scientist,Which scientist developed the theory of genera...,Isaac Newton was an English scientist. Isaac N...,Albert Einstein,Isaac Newton
1,city,In which city can you find the famous Louvre M...,London is a city in England. London has many f...,Paris,London
2,explorer,Which explorer is credited with discovering th...,Marco Polo was an Italian explorer. Marco Polo...,Christopher Columbus,Marco Polo
3,composer,Which composer wrote the famous symphony known...,Wolfgang Amadeus Mozart was a prolific compose...,Ludwig van Beethoven,Wolfgang Amadeus Mozart
4,country,Which country is known for its iconic Sydney O...,Canada is a country in North America. Canada i...,Australia,Canada


### Models for Huggingface

In [27]:
# llama 2
LLAMA2_70B_CHAT_HF = 'meta-llama/Llama-2-70b-chat-hf' 
LLAMA2_13B_CHAT_HF = 'meta-llama/Llama-2-13b-chat-hf'

# mistral
MIXTRAL_8X7B_INSTRUCT_V01_HF = 'mistralai/Mixtral-8x7B-Instruct-v0.1'
MISTRAL_INSTRUCT_V02_HF = 'mistralai/Mistral-7B-Instruct-v0.2'

# gemma
GEMMA_7B_IT_HF = "google/gemma-7b-it"

## Prompt Templates

### 1. Create Prompt Templates

Three different settings:
- Oracle: Zero-shot prompting using zero-attractor sentences only (i.e., no irrelevant context contained) (~ upper-bound)
- Baseline: Zero-shot prompting  using a simple system prompt to describe the task and sentences containing irrelevant context (~ lower-bound) 
- S2A: two-step zero-shot prompting technique that first prompts the model to extract irrelevant context and then conditions on the response to generate final answer.

In [ ]:
# following W&S (2023), baseline, oracle and s2a step 2 share the same system prompt to generate the final answer; the system prompt is loosely based on W&S (2023) Fig. 16
system_prompt = 'Complete the following sentence with one single word: '
#system_prompt_comps = "State whether the following sentence is true or false. Separate into two categories labeled with 'Reasoning:' and 'Final answer (true or false):' \nSentence: "
system_prompt_comps = "Is the conclusion of the following statement true or false? Just answer with 'True' or 'False'. Disregard any potential nonce words for your answer. \nStatement: " # consider adding instructions regarding the nonce words
#system_prompt_comps = "Is the conclusion of the following statement true or false? Simply answer with 'True' or 'False'. \nStatement: " # consider adding instructions regarding the nonce words
system_prompt = "Given the following premises, please determine for which one the conclusion can be drawn from:\n\n"

# s2a step 1 prompt is loosely based on W&S (2023), Fig. 15
fig_15 = '''Given the following text by a user, extract the part that is related and useful, so that
using that text alone would be good context for providing an accurate and correct answer
to the question portion of the text. Please include the actual question or query that the
user is asking. Separate this into two categories labeled with ”Context text related to the
question (includes all content except unrelated sentences):” and ”Detailed question:”. Do
not use list.'''

# step 1: separate relevant from distracting information 
if dataset == 'noise':
    label = "Context text related to the incomplete sentence (includes all content except unrelated sentences):"
    s2a_step1 = f'''Given the following incomplete sentence, extract the part that is related and useful, so that using that text alone would a be good context for providing an accurate completion to the incomplete sentence. Label your response with "{label}".
    
    Text by user: '''

if dataset == 'comps':
    label = "Context relevant to conlcusion (includes all content except irrelevant sentences):"
    s2a_step1 = f'''Given the following text by a user, extract the part that is related and useful, so that using that text alone would be sufficient context for stating whether the conclusion is true or false. Label your response with "{label}". \nText by user: '''

if dataset in ['qa', 'qa_ic']:
    label = "Question (does not include irrelevant context):"
    s2a_step1 = f'''Given the following text by a user, remove the the portion that is irrelevant to answer the question. Label your response with "{label}".
    
    Text by User: '''


In [29]:
system_prompt

'Given the following text by a user, extract the part that is related and useful, so that using that text alone would be sufficient context for stating whether the conclusion is true or false. Label your response with "Context relevant to conlcusion (includes all content except irrelevant sentences):". \nText by user: '

## Experiments with HF

In [29]:
s2a = reload(s2a)
max_memory_mapping = {0: "0GB", 1: "0GB", 2: "0GB", 3:"0GB", 4:"0GB", 5:"36GB", 6:"36GB", 7:"30GB"}
model, tokenizer = s2a.load_model_hf(LLAMA2_70B_CHAT_HF, memory_pinning=max_memory_mapping, in_4bit=True)

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Loading checkpoint shards:  33%|███▎      | 5/15 [01:11<02:22, 14.24s/it]


KeyboardInterrupt: 

: 

In [51]:
# load data
s2a = reload(s2a)
sents, targets = (comps_base['sentence_acc'], comps_base['sentence_unacc']), (comps_base['acceptable_concept'], comps_base['unacceptable_concept'])

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [37]:
# step 1 s2a
data = s2a.preprocess(sents, tokenizer, model.name_or_path, s2a_step1)
print(tokenizer.convert_ids_to_tokens(data['input_ids'][0], skip_special_tokens=True))
#file_spec = f"{dataset}_{data_subset}_s2a_step1_{model_name}"

N = 2
batch_size = 8
dataloader = DataLoader(data, collate_fn=default_data_collator, batch_size=batch_size)
#preds = s2a.generate_hf(model, tokenizer, dataloader, sents, targets, N=N, out=False, file_spec=None)
#sents = s2a.post_proc_s2a(preds, dataset, label)
#file_spec = f"{dataset}_{data_subset}_s2a_step2_{model_name}"

['▁[', 'INST', ']', '▁<<', 'SY', 'S', '>>', '<0x0A>', 'G', 'iven', '▁the', '▁following', '▁text', '▁by', '▁a', '▁user', ',', '▁extract', '▁the', '▁part', '▁that', '▁is', '▁related', '▁and', '▁useful', ',', '▁so', '▁that', '▁using', '▁that', '▁text', '▁alone', '▁would', '▁be', '▁sufficient', '▁context', '▁for', '▁stating', '▁whether', '▁the', '▁conclusion', '▁is', '▁true', '▁or', '▁false', '.', '▁Label', '▁your', '▁response', '▁with', '▁"', 'Context', '▁relevant', '▁to', '▁con', 'l', 'c', 'usion', '▁(', 'includes', '▁all', '▁content', '▁except', '▁irrelevant', '▁sentences', '):', '".', '▁', '<0x0A>', 'Text', '▁by', '▁user', ':', '▁', '<0x0A>', '<', '</', 'SY', 'S', '>>', '<0x0A>', '<0x0A>', 'A', '▁w', 'ug', '▁is', '▁a', '▁muss', 'el', '.', '▁Therefore', ',', '▁a', '▁w', 'ug', '▁attach', 'es', '▁to', '▁rocks', '.', '▁[', '/', 'INST', ']']


In [57]:
# prepare data for step 2 s2a
s2a = reload(s2a)
data = s2a.preprocess(sents, tokenizer, model.name_or_path, system_prompt_comps)

batch_size = 8
dataloader = DataLoader(data, collate_fn=default_data_collator, batch_size=batch_size)
print(tokenizer.convert_ids_to_tokens(data['input_ids'][0], skip_special_tokens=True))
preds = s2a.generate_hf(model, tokenizer, dataloader, sents, targets, N=3) #, out=True, file_spec='multiSem_baseline_oracle_Llama-2-70b-chat-hf')
preds

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


['▁[', 'INST', ']', '▁<<', 'SY', 'S', '>>', '<0x0A>', 'Is', '▁the', '▁conclusion', '▁of', '▁the', '▁following', '▁statement', '▁true', '▁or', '▁false', '?', '▁Just', '▁answer', '▁with', "▁'", 'True', "'", '▁or', "▁'", 'False', "'.", '▁Dis', 'reg', 'ard', '▁any', '▁potential', '▁non', 'ce', '▁words', '▁for', '▁your', '▁answer', '.', '▁', '<0x0A>', 'Statement', ':', '▁', '<0x0A>', '<', '</', 'SY', 'S', '>>', '<0x0A>', '<0x0A>', 'A', '▁w', 'ug', '▁is', '▁a', '▁muss', 'el', '.', '▁Therefore', ',', '▁a', '▁w', 'ug', '▁attach', 'es', '▁to', '▁rocks', '.', '▁[', '/', 'INST', ']']


Batches:   0%|          | 2/1737 [01:10<17:00:44, 35.30s/it]


[' False',
 ' False',
 ' False',
 ' False',
 ' True',
 ' True',
 ' True',
 ' True',
 ' False',
 ' False',
 ' False',
 ' False.\n\nThe statement is trying to trick the reader into assuming that "blicket',
 ' True',
 ' True',
 ' True',
 ' True',
 ' False.\n\nThe statement "A fep is a crocodile" is not true',
 ' True',
 ' False',
 ' True',
 ' True',
 ' False',
 ' True',
 ' False']

: 

## Evaluation

In [21]:
import pandas as pd
from importlib import reload
import s2a

s2a = reload(s2a)
path = '/local/js/lmcm_project/eval/comps_before_baseline_oracle_llama2-70.csv'
eval_df = pd.read_csv(path)
preds, targets = eval_df['prediction'], eval_df['target']
#print(["Final answer (true or false):" in pred for pred in preds])
s2a.accuracy(preds, targets, comps=True)


Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


False

Explanation:
The sentence is false because the conclusion "A wug attaches to rocks" does not logically follow from the premises. While it is true that some clams and mussels can attach to rocks, it is not a characteristic that is unique to these two types of shellfish. Many other types of shellfish, such as barnacles and oysters, can also attach to rocks. Therefore, the deduction is not valid, and the sentence is false.
False
False
False
False. The sentence is a play on words and does not convey a valid statement.
False
False
True
contained


0.12

In [55]:
s2a = reload(s2a)
file_spec = "singleSem_baseline_oracle_llama2-70"
path = f"/local/js/lmcm_project/eval/{file_spec}.csv"
eval_df = pd.read_csv(path)
preds, targets = eval_df['prediction'], eval_df['target']
preds = s2a.post_proc(preds)
acc_isin = s2a.accuracy(preds, targets, isin=True)
acc_em = s2a.accuracy(preds, targets, exact_match=True)
acc_fw = s2a.accuracy(preds, targets, first_word=True)

print(f"Is-in Accuracy:\t\t{acc_isin} \nExact Match Accuracy:\t{acc_em} \nFirst Word Accuracy:\t{acc_fw}")

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Is-in Accuracy:		0.92 
Exact Match Accuracy:	0.92 
First Word Accuracy:	0.92


## Random stuff

In [ ]:
four_bit_memory = 4551360512
with_acc_4bit = 7241732096
mistral_wo_quant = 30040678400
llama_memory = 26462937088
mistral_wo_quant/llama_memory